<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/100_CO%C4%B0N.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

API_KEY = "APIKEY"

def get_top_coins(limit=100):  # limit'i 100 yapıyoruz
    url = f"https://min-api.cryptocompare.com/data/top/mktcapfull"
    headers = {'authorization': f'Apikey {API_KEY}'}
    params = {'limit': limit, 'tsym': 'USDT'}
    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:  # Eğer API'den 200 OK yanıtı gelmezse
        print(f"Hata: API yanıtı alınamadı ({response.status_code})")
        return []

    data = response.json().get('Data', [])
    return [coin['CoinInfo']['Name'] for coin in data]

def get_historical_data(symbol, timeframe='hour', limit=500):
    url = f"https://min-api.cryptocompare.com/data/v2/histo{timeframe}"
    headers = {'authorization': f'Apikey {API_KEY}'}
    params = {'fsym': symbol, 'tsym': 'USDT', 'limit': limit}
    response = requests.get(url, params=params, headers=headers)

    if response.status_code != 200:  # API yanıtını kontrol et
        print(f"Hata: {symbol} verisi alınamadı ({response.status_code})")
        return pd.DataFrame()  # Boş dataframe döndür

    df = pd.DataFrame(response.json().get('Data', {}).get('Data', []))
    if df.empty:
        print(f"Hata: {symbol} için veri bulunamadı.")
        return df

    df['timestamp'] = pd.to_datetime(df['time'], unit='s')
    return df

def calculate_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)

    ma_up = up.ewm(com=period-1, adjust=True, min_periods=period).mean()
    ma_down = down.ewm(com=period-1, adjust=True, min_periods=period).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100 / (1 + rsi))
    return rsi

def calculate_c20_signals(data, symbol):
    data = data.copy()
    data['symbol'] = symbol
    data['logClose'] = np.log(data['close'])
    data['rsi'] = calculate_rsi(data['logClose'], 14)
    data['rsiChange'] = data['rsi'].diff()

    data['prev_rsiChange'] = data['rsiChange'].shift(1)
    data['c20_long'] = (data['rsiChange'].gt(20)) & (data['prev_rsiChange'].le(20))
    data['c20_short'] = (data['rsiChange'].lt(-20)) & (data['prev_rsiChange'].ge(-20))

    return data

def process_4h_data(df):
    df = df.set_index('timestamp')
    df_4h = df.resample('4H').agg({
        'time': 'first',
        'close': 'last',
        'high': 'max',
        'low': 'min',
        'open': 'first',
        'volumefrom': 'sum'
    }).dropna()
    return df_4h.reset_index()

def main():
    top_coins = get_top_coins(100)  # İlk 100 coin çekiyoruz
    if not top_coins:
        print("Coin listesi alınamadı.")
        return

    print(f"Analiz edilecek coinler: {', '.join(top_coins)}\n")

    # Son iki ayın tarihini hesapla
    two_months_ago = datetime.now() - timedelta(days=60)

    for symbol in top_coins:
        print(f"\n{symbol} için sinyaller:")
        try:
            # 4H data (son 1 gün)
            df_4h = get_historical_data(symbol, 'hour', 2000)
            if df_4h.empty:
                continue
            df_4h = df_4h[df_4h['timestamp'] > two_months_ago]  # Son iki aydan önceki verileri çıkar
            df_4h = process_4h_data(df_4h)
            signals_4h = calculate_c20_signals(df_4h, symbol)

            # Günlük data (son 2 ay)
            df_daily = get_historical_data(symbol, 'day', 500)
            if df_daily.empty:
                continue
            df_daily = df_daily[df_daily['timestamp'] > two_months_ago]  # Son iki aydan önceki verileri çıkar
            signals_daily = calculate_c20_signals(df_daily, symbol)

            # 4H sinyalleri
            signal_4h = signals_4h[signals_4h['c20_long'] | signals_4h['c20_short']]
            if not signal_4h.empty:
                print("\nSon 1 Günlük 4H Sinyaller:")
                signal_4h.apply(
                    lambda x: print(f"Tarih: {x['timestamp']}, "
                                f"Sinyal: {'LONG' if x['c20_long'] else 'SHORT'}, "
                                f"Fiyat: {x['close']:.2f}, "
                                f"RSI Değişim: {x['rsiChange']:.2f}"),
                    axis=1)

            # Günlük sinyaller
            signal_daily = signals_daily[signals_daily['c20_long'] | signals_daily['c20_short']]
            if not signal_daily.empty:
                print("\nSon 2 Aydaki Günlük Sinyaller:")
                signal_daily.apply(
                    lambda x: print(f"Tarih: {x['timestamp']}, "
                                f"Sinyal: {'LONG' if x['c20_long'] else 'SHORT'}, "
                                f"Fiyat: {x['close']:.2f}, "
                                f"RSI Değişim: {x['rsiChange']:.2f}"),
                    axis=1)

        except Exception as e:
            print(f"Hata: {symbol} için veri alınamadı - {str(e)}")
            continue

if __name__ == "__main__":
    main()


Analiz edilecek coinler: BTC, ETH, XRP, USDT, SOL, BNB, DOGE, USDC, SUI, ADA, STETH, TON, RYO, TRX, WLD, LINK, XLM, AVAX, ENA, CULT, HBAR, ONDO, UNI, SHIB, WBTC, BGB, DOT, APT, MOVE, BCH, WBT, PEPE, JUP, LEO, LTC, MANTLE, OP, ARB, OM, WEETH, NEAR, EIGEN, USDE, ICP, TIA, AAVE, ZRO, GT, STRK, POL, MATIC, CRO, VIRTUAL, ETC, VET, XMR, NGTG, RNDR, TAO, SEI, PYTH, MORPHO, FET, DAI, ALGO, ENS, JTO, ME, FIL, KAS, W, RENDER, FLR, BONK, WBETH, RAY, ATOM, IMX, ATH, GRASS, MOCA, STX, HTX, IO, ZK, PEAQ, CORE, THETA, FTM, FDUSD, GRT, ETHFI, CRV, AI16Z, RONIN, INJ, AERO, VANA, ZEDXION, RUNE


BTC için sinyaller:


<ipython-input-1-d9673e60a029>:66: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



Son 1 Günlük 4H Sinyaller:
Tarih: 2024-11-06 00:00:00, Sinyal: LONG, Fiyat: 74343.96, RSI Değişim: 24.34

ETH için sinyaller:


<ipython-input-1-d9673e60a029>:66: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



Son 1 Günlük 4H Sinyaller:
Tarih: 2024-11-06 00:00:00, Sinyal: LONG, Fiyat: 2579.36, RSI Değişim: 29.39
Tarih: 2024-12-20 12:00:00, Sinyal: LONG, Fiyat: 3374.68, RSI Değişim: 22.46

XRP için sinyaller:


...

<ipython-input-1-d9673e60a029>:66: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



BNB için sinyaller:


<ipython-input-1-d9673e60a029>:66: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({



DOGE için sinyaller:


<ipython-input-1-d9673e60a029>:66: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_4h = df.resample('4H').agg({
